# Initialization and package preparation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Data load and preprocessing

In [15]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
y_train

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.2, random_state=rso)

X_train, y_train = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)
X_dev, y_dev = X_train[:10000,:], y_train[:10000]

# Load back neural network and give it a try

In [7]:
with open('search_lr_wd.pkl', 'rb') as f:
    neural_outcome = pickle.load(f)

In [22]:
import torch 
from torch.autograd import Variable
dtype = torch.FloatTensor


nn_model = neural_outcome['best_net']
nn_model.net.eval()
# X_train_test_tensor = torch.from_numpy(X_train_test.values)
# X_train_test_tensor= X_train_test_tensor.type(dtype)
# out = nn_model.test(X_train_test_tensor)
# nn_model.check_auc(out,y_train_test)


idx = test_data['id']
test_data_mutated = test_data.drop(['id'], axis=1)
test_data_mutated = naive_pre.dtype_change(test_data_mutated)
test_data_mutated = naive_pre.scale(test_data_mutated, test=True)
test_data_tensor = torch.from_numpy(test_data_mutated.values)
test_data_tensor = test_data_tensor.type(dtype)
out = nn_model.test(test_data_tensor)

In [44]:
result= np.hstack((idx.values.reshape(892816,1), out.data.numpy()[:,1].reshape(892816,1)))
print(result.shape)
result[:,0] = result[:,0].astype(np.int32)
np.savetxt('submission.csv',result, fmt=['%d', '%.9f'] , delimiter= ',', header= 'id, target')

(892816, 2)


In [ ]:
# ! cat submission.csv 10
#! sudo pip3 install lightgbm

# Try to use Lightgbm here.

In [2]:
! svn export https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide

/bin/sh: 1: svn: not found


In [79]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
y_train

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.2, random_state=rso)

X_train, y_train, col = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)
X_dev, y_dev = X_train[:10000,:], y_train[:10000]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:

train_data11 = data_util.load_train_data()
train_data11.head(10)
# for col in train_data11.columns:
#     if 'bin' in col or 'cat' in col:
#         train_data11[col] = train_data11[col].astype('category')
# data = pd.get_dummies(train_data11)
# data.columns

uselessones = ['ps_ind_14',
             'ps_ind_10_bin',
             'ps_ind_11_bin',
             'ps_ind_12_bin', 
             'ps_ind_13_bin',
             'ps_ind_18_bin',
             'ps_car_10_cat',
             'ps_calc_15_bin',
             'ps_calc_20_bin'
        ]
print(train_data11.columns)
train_data11 = train_data11.drop(uselessones, axis =1)
print('hhhhh')
print(train_data11.columns)

# If increase data will boost my performance??
Well, not really, at least insignificantly.

In [27]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 
from sklearn import metrics
import lightgbm as lgb



rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
lv1_pre = data_preprocess.preprocess_cell()
X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']

X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size =0.2, random_state=rso)

X_train, y_train, col = lv1_pre.process(X_train, y=y_train, rso =rso)
X_val = lv1_pre.process(X_val,test=True,rso =rso)
X_train_test = lv1_pre.process(X_train_test,test=True,rso =rso)

auc_dict={}

lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train, free_raw_data=False)

#seems like lambda_l2 equals 0.0342
lgb1_params = {
    'boosting_type':'gbdt',
    'objective':'binary',
    'metric': 'binary_logloss',
    'lambda_l2':0.0342,
    'learning_rate': 0.09, 
    'feature_fraction': 0.9,   #
    'bagging_fraction': 0.8,   #
    'bagging_freq': 10,
    'verbose': 1,
    'verbose_eval':False
}
for i in range(1):
    
    #seems like the best combination is learning rate:0.1, Weight decay:0.05
    lr = 10**np.random.uniform(-2,0)
    #weight_decay = 10** np.random.uniform(-4,2)
    weight_decay = 10000000
    lgb1_params['learning_rate'] = lr
    lgb1_params['lambda_l2'] = weight_decay
    lgb1 = lgb.train(lgb1_params,
            lgb_train,
            num_boost_round =2500,  #Number of trees it will generate  
            valid_sets= lgb_val,
        early_stopping_rounds = 5
            )

    y_pred1 = lgb1.predict(X_train_test, lgb1.best_iteration)
    score = metrics.roc_auc_score(y_train_test,y_pred1)
    describe = '{}: We reach {} with lr {}, wd {} at the {} round!'.format(i+1, score,lr,weight_decay, lgb1.best_iteration)
    print(describe)
    auc_dict[describe] = score

[1]	valid_0's binary_logloss: 0.693147
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.693147
[3]	valid_0's binary_logloss: 0.693147
[4]	valid_0's binary_logloss: 0.693147
[5]	valid_0's binary_logloss: 0.693147
[6]	valid_0's binary_logloss: 0.693147
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693147
1: We reach 0.5 with lr 0.16928286101010034, wd 10000000 at the 1 round!


In [45]:
for i in sorted(auc_dict, key = lambda x: auc_dict[x], reverse=True):
    print(i)

71: We reach 0.6425146075906395 with lr 0.10090459507003365, wd 0.03426226856475561 at the 102 round!
148: We reach 0.6424326925153181 with lr 0.10745944670147414, wd 0.0003203867371549406 at the 83 round!
65: We reach 0.6421573746817226 with lr 0.04258541643753987, wd 0.21953639465793823 at the 237 round!
35: We reach 0.6421277053525621 with lr 0.013641124580520884, wd 0.6284206847642912 at the 760 round!
93: We reach 0.6421208784246081 with lr 0.014135085650213666, wd 9.483519863230152 at the 640 round!
31: We reach 0.6421177994559204 with lr 0.04477025632952884, wd 0.0010087862148837454 at the 218 round!
116: We reach 0.6419487947034517 with lr 0.01697460315960794, wd 0.0005371775659281679 at the 560 round!
36: We reach 0.6418749961764769 with lr 0.0140269328712022, wd 0.035941098081195236 at the 652 round!
7: We reach 0.6416946508246797 with lr 0.012987316213487771, wd 0.7248514625482201 at the 640 round!
42: We reach 0.6416197641804985 with lr 0.01567690910094347, wd 4.11173258472

In [3]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import numpy as np 

rso = np.random.RandomState(66)

train_data = data_util.load_train_data()
test_data= data_util.load_test_data()

lv1_pre = data_preprocess.preprocess_cell()

X_train = train_data.drop(['id','target'],axis=1)
y_train = train_data['target']
X_test = test_data.drop(['id'], axis=1)

# combs = [
#     ('ps_reg_01', 'ps_car_02_cat'),  
#     ('ps_reg_01', 'ps_car_04_cat'),
# ]

# #'ps_reg_01_plus_ps_car_02_cat'
# #'ps_reg_01_plus_ps_car_04_cat'
# for n_c, (f1, f2) in enumerate(combs):
#     name1 = f1 + "_plus_" + f2

#     X_train[name1] = X_train[f1].apply(lambda x: str(x)) + "_" + X_train[f2].apply(lambda x: str(x))
#     X_test[name1] = X_test[f1].apply(lambda x: str(x)) + "_" + X_test[f2].apply(lambda x: str(x)) 
#     lbl = LabelEncoder()
#     lbl.fit(list(X_train[name1].values) + list(X_test[name1].values))
#     X_train[name1] = lbl.transform(list(X_train[name1].values))
#     X_test[name1] = lbl.transform(list(X_test[name1].values))


X_train, y_train, col = lv1_pre.process(X_train, y=y_train, rso =rso, oversample= None)
X_test = lv1_pre.process(X_test,test=True,rso =rso)

result = pd.DataFrame()
result['id'] = test_data['id']

In [ ]:
from Stacker import Ensemble

tic= datetime.now()
stack_model = Ensemble(n_splits=5)
y_pred= stack_model.fit_predict(X_train,y_train, X_test, META = 'nn')
toc = datetime.now()
print('This is round you consume {} time to run this model.'.format(toc-tic))

result['target'] = y_pred
result.to_csv('submission11.csv', index=False)

[1]	valid_0's binary_logloss: 0.651205
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.613263
[3]	valid_0's binary_logloss: 0.578807
[4]	valid_0's binary_logloss: 0.547396
[5]	valid_0's binary_logloss: 0.51868
[6]	valid_0's binary_logloss: 0.492351
[7]	valid_0's binary_logloss: 0.468154
[8]	valid_0's binary_logloss: 0.445878
[9]	valid_0's binary_logloss: 0.425329
[10]	valid_0's binary_logloss: 0.406339
[11]	valid_0's binary_logloss: 0.388766
[12]	valid_0's binary_logloss: 0.372485
[13]	valid_0's binary_logloss: 0.357377
[14]	valid_0's binary_logloss: 0.343355
[15]	valid_0's binary_logloss: 0.330322
[16]	valid_0's binary_logloss: 0.318198
[17]	valid_0's binary_logloss: 0.306918
[18]	valid_0's binary_logloss: 0.296413
[19]	valid_0's binary_logloss: 0.286623
[20]	valid_0's binary_logloss: 0.277504
[21]	valid_0's binary_logloss: 0.268994
[22]	valid_0's binary_logloss: 0.261055
[23]	valid_0's binary_logloss: 0.253646
[24]	valid_0's binary_logloss

[48]	valid_0's binary_logloss: 0.168465
[49]	valid_0's binary_logloss: 0.167275
[50]	valid_0's binary_logloss: 0.16617
[51]	valid_0's binary_logloss: 0.165141
[52]	valid_0's binary_logloss: 0.164187
[53]	valid_0's binary_logloss: 0.16331
[54]	valid_0's binary_logloss: 0.162496
[55]	valid_0's binary_logloss: 0.16173
[56]	valid_0's binary_logloss: 0.161021
[57]	valid_0's binary_logloss: 0.160373
[58]	valid_0's binary_logloss: 0.159761
[59]	valid_0's binary_logloss: 0.159195
[60]	valid_0's binary_logloss: 0.158683
[61]	valid_0's binary_logloss: 0.158208
[62]	valid_0's binary_logloss: 0.157765
[63]	valid_0's binary_logloss: 0.157352
[64]	valid_0's binary_logloss: 0.156966
[65]	valid_0's binary_logloss: 0.156611
[66]	valid_0's binary_logloss: 0.156286
[67]	valid_0's binary_logloss: 0.155982
[68]	valid_0's binary_logloss: 0.155704
[69]	valid_0's binary_logloss: 0.155441
[70]	valid_0's binary_logloss: 0.155206
[71]	valid_0's binary_logloss: 0.154989
[72]	valid_0's binary_logloss: 0.154794
[73

[75]	valid_0's binary_logloss: 0.154297
[76]	valid_0's binary_logloss: 0.154153
[77]	valid_0's binary_logloss: 0.154015
[78]	valid_0's binary_logloss: 0.153885
[79]	valid_0's binary_logloss: 0.153766
[80]	valid_0's binary_logloss: 0.15366
[81]	valid_0's binary_logloss: 0.153559
[82]	valid_0's binary_logloss: 0.153468
[83]	valid_0's binary_logloss: 0.153384
[84]	valid_0's binary_logloss: 0.15331
[85]	valid_0's binary_logloss: 0.153239
[86]	valid_0's binary_logloss: 0.153167
[87]	valid_0's binary_logloss: 0.153096
[88]	valid_0's binary_logloss: 0.153041
[89]	valid_0's binary_logloss: 0.152986
[90]	valid_0's binary_logloss: 0.15294
[91]	valid_0's binary_logloss: 0.152883
[92]	valid_0's binary_logloss: 0.152841
[93]	valid_0's binary_logloss: 0.152794
[94]	valid_0's binary_logloss: 0.152747
[95]	valid_0's binary_logloss: 0.152707
[96]	valid_0's binary_logloss: 0.152674
[97]	valid_0's binary_logloss: 0.152643
[98]	valid_0's binary_logloss: 0.152605
[99]	valid_0's binary_logloss: 0.152567
[10

[130]	valid_0's binary_logloss: 0.151903
[131]	valid_0's binary_logloss: 0.151902
[132]	valid_0's binary_logloss: 0.151893
[133]	valid_0's binary_logloss: 0.151884
[134]	valid_0's binary_logloss: 0.151879
[135]	valid_0's binary_logloss: 0.151874
[136]	valid_0's binary_logloss: 0.151871
[137]	valid_0's binary_logloss: 0.151856
[138]	valid_0's binary_logloss: 0.151855
[139]	valid_0's binary_logloss: 0.151857
[140]	valid_0's binary_logloss: 0.151855
[141]	valid_0's binary_logloss: 0.151839
[142]	valid_0's binary_logloss: 0.151822
[143]	valid_0's binary_logloss: 0.151818
[144]	valid_0's binary_logloss: 0.151805
[145]	valid_0's binary_logloss: 0.151797
[146]	valid_0's binary_logloss: 0.151792
[147]	valid_0's binary_logloss: 0.151781
[148]	valid_0's binary_logloss: 0.151779
[149]	valid_0's binary_logloss: 0.151776
[150]	valid_0's binary_logloss: 0.151781
[151]	valid_0's binary_logloss: 0.151779
[152]	valid_0's binary_logloss: 0.15177
[153]	valid_0's binary_logloss: 0.151763
[154]	valid_0's b

[164]	valid_0's binary_logloss: 0.151943
[165]	valid_0's binary_logloss: 0.151937
[166]	valid_0's binary_logloss: 0.151939
[167]	valid_0's binary_logloss: 0.151933
[168]	valid_0's binary_logloss: 0.151932
[169]	valid_0's binary_logloss: 0.151928
[170]	valid_0's binary_logloss: 0.151921
[171]	valid_0's binary_logloss: 0.151917
[172]	valid_0's binary_logloss: 0.151914
[173]	valid_0's binary_logloss: 0.151911
[174]	valid_0's binary_logloss: 0.151902
[175]	valid_0's binary_logloss: 0.151898
[176]	valid_0's binary_logloss: 0.151896
[177]	valid_0's binary_logloss: 0.151892
[178]	valid_0's binary_logloss: 0.15189
[179]	valid_0's binary_logloss: 0.151892
[180]	valid_0's binary_logloss: 0.151887
[181]	valid_0's binary_logloss: 0.151885
[182]	valid_0's binary_logloss: 0.151891
[183]	valid_0's binary_logloss: 0.15189
[184]	valid_0's binary_logloss: 0.151884
[185]	valid_0's binary_logloss: 0.151883
[186]	valid_0's binary_logloss: 0.151883
[187]	valid_0's binary_logloss: 0.151884
[188]	valid_0's bi

...

Epoch 0: iteration 0, the loss is [ 0.88721603]
  acc for train: 0.9635541622144714, acc for val: 0.96327
  auc for train: 0.6129201870987113, auc for val: 0.6098416438698622
--------------------------------------------------------------
Epoch 0: iteration 100, the loss is [ 0.20654897]
  acc for train: 0.963552482140817, acc for val: 0.96329
  auc for train: 0.6410678220498842, auc for val: 0.6427860747897316
--------------------------------------------------------------
Epoch 0: iteration 200, the loss is [ 0.18505025]
  acc for train: 0.963552482140817, acc for val: 0.96329
  auc for train: 0.6330309870743006, auc for val: 0.6340475471276239
--------------------------------------------------------------
Epoch 0: iteration 300, the loss is [ 0.16718414]
  acc for train: 0.963552482140817, acc for val: 0.96329
  auc for train: 0.6411116265349107, auc for val: 0.6426830613493931
--------------------------------------------------------------
Epoch 0: iteration 400, the loss is [ 0.13806

Process Process-1361:
Traceback (most recent call last):
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 34, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()


BLOWINGUP: Learning rate is 0.020576266888179925. Weight decay is 0.0038547135436206204. dropout is 0.5
 Val aus is 0.6229894779213635. Train auc is 0.6212915788678677
This is round you consume 0:22:37.933934 time to run this model.


KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception, closing connection.
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/tornado/stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 637, in _abort_queues
    self._abort_queue(stream)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelbase.py", line 662, in _abort_queue
    poll

In [17]:
#! ls -a
# ! rm 
# X_train.describe()
for i in sorted(stack_model.train_hist, key =lambda x:x[0], reverse=True):
    print(stack_model.train_hist[i])


Learning rate is 0.0006241303598307956. Weight decay is 0.0026706282114709093. dropout is 0.5
 Val aus is 0.6431840995163449. Train auc is 0.6409540445249023
Learning rate is 0.0004130048057886514. Weight decay is 0.015377794660046963. dropout is 0.5
 Val aus is 0.6429664246626596. Train auc is 0.6408428527176809
Learning rate is 0.025221516643856644. Weight decay is 0.015984258287073554. dropout is 0.5
 Val aus is 0.6429169441072538. Train auc is 0.6408618952542339
Learning rate is 0.0055353144032044485. Weight decay is 0.020228773910792964. dropout is 0.5
 Val aus is 0.6428881663463115. Train auc is 0.6406036645498829
Learning rate is 0.020576266888179925. Weight decay is 0.0038547135436206204. dropout is 0.5
 Val aus is 0.6428809043908161. Train auc is 0.6410302614713866
Learning rate is 0.001341993635972462. Weight decay is 0.044122990686213025. dropout is 0.5
 Val aus is 0.6428361774752811. Train auc is 0.6406958356800573
Learning rate is 0.0005067940145389166. Weight decay is 0.0

In [17]:
#result.to_csv('submission04.csv', index=False)
# train_data.shape
# X_train.shape
# np.bincount(y_train)
# ! pwd
miscellany
X_test.head(10)

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,...,ps_calc_16_bin_0,ps_calc_16_bin_1,ps_calc_17_bin_0,ps_calc_17_bin_1,ps_calc_18_bin_0,ps_calc_18_bin_1,ps_calc_19_bin_0,ps_calc_19_bin_1,ps_calc_20_bin_0,ps_calc_20_bin_1
0,-0.957955,1.324746,-0.097621,1.325445,-0.385866,-0.344291,-9.076669e-01,-1.616955,-1.092966,-0.639875,...,0,1,0,1,1,0,1,0,0,1
1,1.058390,0.213594,-0.097621,-0.648589,1.004750,0.150435,-3.924898e-01,-1.616955,-1.092966,-0.921442,...,1,0,0,1,0,1,1,0,0,1
2,1.562477,-0.527174,-0.097621,0.761435,-0.733520,-1.086380,7.078733e-02,0.785475,0.343963,0.369450,...,1,0,1,0,1,0,1,0,1,0
3,-0.957955,0.583978,-0.097621,-0.930594,-1.776482,-0.591654,-2.450736e-14,-0.415740,-0.099167,-0.717556,...,1,0,0,1,1,0,1,0,1,0
4,1.562477,0.954362,-0.097621,-0.930594,1.004750,-0.096928,-2.440197e-01,0.785475,-0.099167,-0.001560,...,0,1,0,1,1,0,1,0,0,1
5,-0.957955,0.583978,-0.097621,0.761435,1.004750,0.150435,-4.290506e-01,-0.415740,-1.092966,-0.277586,...,1,0,0,1,1,0,0,1,1,0
6,-0.957955,-0.527174,-0.097621,1.043440,-1.776482,-0.839017,-2.450736e-14,-0.415740,-1.092966,-0.394927,...,0,1,1,0,1,0,1,0,1,0
7,-0.957955,-1.638327,-0.097621,-0.084579,1.004750,1.634614,8.057727e-01,0.785475,-1.092966,-1.009201,...,0,1,1,0,1,0,1,0,1,0
8,-0.957955,0.954362,-0.097621,-0.366584,-0.733520,-1.086380,-9.490565e-01,-0.415740,1.153809,3.681776,...,1,0,0,1,1,0,1,0,1,0
9,-0.453868,0.583978,-0.097621,-0.084579,1.004750,1.387251,-2.450736e-14,-0.415740,1.153809,1.015509,...,0,1,0,1,1,0,1,0,1,0


# Only include LB score in this chunk

#### 10.28
1. No oversample, No variable dropping: 2.75
2. 0.2 SMOTE oversample, No variable dropping: 2.76
3. Take fold outside(ensemble), only fold one, 0.2 SMOTE oversample, No variable dropping: 2.76 (no improvement)
---
#### 10.29
4. 0.4 SMOTE oversample, No variable dropping: 2.7
5. META: Logistic Version, 0.3: 2.56(Don't know what is wrong here)
6. 0.1 SMOTE oversample, No variable dropping: 2.75
7. 0.15 SMOTE oversample, add L2:0.034, No variable dropping: 2.75

#### 10.30
###### (correct the mistake in stacker, forget to take mean for S_test) 
8. 0.22 SMOTE oversample, no META: 0.278
9. 0.20 SMOTE oversample, no META: 0.279 Stacker score: 0.94613
10. 0.25 SMOTE oversample, no META: 0.279 Stacker score: ***
11. 0.18 SMOTE oversample, no META: 0.2 Stacker score: 0.9
###### (won't normalize binary and categorical anymore  
12. 0.18 SMOTE oversample, no META: 0.2 Stacker score: 0.9

###### () 
9. 

# Only include offline test score(auc) in this chunk

#### 10.29

all based on rso66 with 0.1 test set splitted from train_data,ensemble: three lgb in one.
0. As I rememberred, 0.2 SMOTE oversample, No variable dropping: 0.6424
1. 0.4 SMOTE oversample, No variable dropping: 0.641637
2. 0.6 SMOTE oversample, No variable dropping: 0.639938
3. 0.1 SMOTE oversample, No variable dropping: 0.6431368
4. 0.0 SMOTE oversample, No variable dropping: 0.4736901 #what's wrong with it here??
5. 0.3 SMOTE oversample, No variable dropping: 0.641200
6. META: Logistic Version: (0.1, 0.2, 0.3, 0.5, 0.7, 1)
        (0.633452, 0.632539, 0.645008, 0.628007, 0.627307, 0.625253)

In [ ]:
import data_util
import data_preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np 
from lgb_stacker import lgb_ensemble
from Stacker import Ensemble
from sklearn import metrics
#from sklearn.neighbors import KNeighborsClassifier

rso = np.random.RandomState(66)

result = []
for os in [None]:
    train_data = data_util.load_train_data()
    test_data= data_util.load_test_data()
    
    lv1_pre = data_preprocess.preprocess_cell()
    
    X_train = train_data.drop(['id','target'],axis=1)
    y_train = train_data['target']
    
    X_train, X_train_test, y_train, y_train_test = train_test_split(X_train,y_train,test_size =0.1 ,random_state=rso)

    X_train_af, y_train_af, col = lv1_pre.process(X_train, y=y_train, oversample=os, rso =rso)
    X_train_test_af = lv1_pre.process(X_train_test,test=True,rso =rso)
    
    print(type(X_train_af), type(y_train_af), type(X_train_test_af))
    stack_model = Ensemble(n_splits=5)
    y_pred= stack_model.fit_predict(X_train_af, y_train_af, X_train_test_af)

    result.append(metrics.roc_auc_score(y_train_test,y_pred))

In [11]:
from sklearn import metrics
metrics.roc_auc_score(y_train_test,y_pred)
result

[0.64313174736945844, 0.59310226899327134]

# change to the standard lgb

In [ ]:
from Stacker import Ensemble

tic= datetime.now()
stack_model = Ensemble(n_splits=5)
y_pred= stack_model.fit_predict(X_train,y_train, X_test)
toc = datetime.now()
print('This is round you consume {} time to run this model.'.format(toc-tic))

result['target'] = y_pred
result.to_csv('submission08.csv', index=False)
stack_model.miscellany